In [ ]:
list_authors = [3888979]


In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException ,TimeoutException,NoSuchWindowException
from bs4 import BeautifulSoup
import time
import pandas as pd
import csv
import re

# Spécifiez le chemin vers EdgeDriver et initialisez le navigateur Edge en utilisant ce service
driver = webdriver.Edge(service=Service(executable_path=r"C:\Users\Electro Fatal\Desktop\file\edgedriver_win64\msedgedriver.exe"))

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

try:
    # Accédez à la page de connexion
    driver.get("https://www.webofscience.com.eressources.imist.ma/")  # Remplacez par l'URL correcte

    # Attendre que les champs email et mot de passe soient visibles
    email_field = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, "email"))
    )
    password_field = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.ID, "password"))
    )

    # Entrer les informations d'identification
    email_field.send_keys('israa.boudda@usms.ac.ma')
    password_field.send_keys('')
    password_field.send_keys(Keys.RETURN)

except TimeoutException:
    print("Les champs d'email ou de mot de passe n'ont pas été trouvés.")

In [ ]:
wait = WebDriverWait(driver, 10)

def get_author_information(id):
    driver.get(f"https://www.webofscience.com.eressources.imist.ma/wos/author/record/{id}")
    # driver.get(f"https://www.webofscience.com/wos/author/record/{id}")
    time.sleep(5)
    scroll_slowly(driver)


   # Vérifier si la page de l'auteur est introuvable
    try:
        if "authorNotFound" in driver.current_url:
            print(f"L'auteur avec l'ID {id} est introuvable.")
            return None, []  # Retourner None pour l'auteur introuvable
    except NoSuchElementException:
        pass  # Si l'élément d'erreur n'est pas trouvé, continuez normalement
    try:
        infos = {
            'ID de l\'Auteur' : id,
            'nom_complet' : None,
            'pays_affiliation' : None,
            'co_auteurs' : 0,
            'H-Index' : 0,
            "FWCI": "0",
            'Sum of Times Cited' : 0
        }

        # Récupération des co-auteurs
        try:
            co_auteurs = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'authors-list-link')))
            infos['co_auteurs'] = [auteur.text for auteur in co_auteurs]
            id_co_auteur = [auteur.get_attribute('href').split('/')[-1] for auteur in co_auteurs]
        except:
            infos['co_auteurs'] = []
            id_co_auteur = []
            print('Les co-auteurs n\'existent pas')

        # Récupération des métriques
        try:
            metric_descriptor = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'wat-author-metric-descriptor')))
            for metric in metric_descriptor:
                if metric.text in ['H-Index', 'Sum of Times Cited']:
                    value = metric.find_element(By.XPATH, './preceding-sibling::div').text
                    infos[metric.text] = value
        except:
            print('H-index ou bien citation n\'existent pas')

        # Récupération du pays d'affiliation
        try:
            value = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'more-details'))).text.split(',')[-1].strip()
            infos['pays_affiliation'] = value
        except:
            print('Pays d\'affiliation n\'existe pas')

        # Récupération du nom complet
        try:
            value = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'wat-author-name'))).text
            infos['nom_complet'] = value
        except:
            print('Le nom n\'existe pas')

        return infos , id_co_auteur
    except:
        print('Erreur lors de l\'attente du chargement de la page')

def extract_article_details(driver, article_link):
    driver.get(article_link)
    time.sleep(4)  # Pause pour laisser le temps à la page de se charger
    scroll_slowly(driver)  # Faire défiler la page lentement pour charger le contenu
    infos = {}  # Dictionnaire pour stocker les informations de l'article


    # Titre de l'article
    try:
        infos['Titre de l’article'] = wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, 'title'))
        ).text
    except Exception as e:
        infos['Titre de l’article'] = None
        print("Erreur pour Titre de l’article:")

    # Auteur
    try:
        # Récupérer les noms des auteurs ou les mots-clés
        authors = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[starts-with(@id,'SumAuthTa-DisplayName-author-en-')]")))
        infos['Auteurs'] = ' ; '.join([element.text for element in authors])  # Joindre les noms
    except Exception as e:
        infos['Auteur'] = None
        print("Erreur pour Auteur:" )

    # Date de publication
    class_names = ['FullRTa-pubdate', 'FullRTa-earlyAccess']
    for class_name in class_names:
        try:
            infos['Date de publication'] = wait.until(EC.presence_of_element_located((By.ID, class_name))).text
            break  # Sortir de la boucle si trouvé
        except Exception as e:
            infos['Date de publication'] = None
            print("Erreur pour date:" )

    # Source
    """class_names = ['summary-source-title-link', 'summary-source-title']
    for class_name in class_names:
        try:
            infos['Nom journal'] = wait.until(EC.presence_of_element_located((By.CLASS_NAME, class_name))).text
            break  # Sortir de la boucle si trouvé
        except Exception as e:
            infos['nom journal'] = None
            print("Erreur pour Source:" )"""
            # Source
    class_names = ['summary-source-title-link', 'summary-source-title']
    for class_name in class_names:
        try:
            journal_text = wait.until(EC.presence_of_element_located((By.CLASS_NAME, class_name))).text
            infos['nom journal'] = journal_text.replace("arrow_drop_down", "").strip()  # Supprimer "arrow_drop_down" et les espaces
            break  # Sortir de la boucle si trouvé
        except Exception as e:
            infos['nom journal'] = None
        print("Erreur pour Source:")
    # Mots-clés
    mots_cles = ''
    id_names = ['FRkeywordsTa-keyWordsPlusLink-','FRkeywordsTa-authorKeywordLink-']
    for id_name in id_names:
        try:
            value = wait.until(EC.presence_of_all_elements_located((By.XPATH, f"//a[starts-with(@id,'{id_name}')]")))
            mots_cles = mots_cles +' ' + ' ; '.join([element.text for element in value])  # Joindre les noms
        except Exception as e:
            infos['Mots-clés'] = None
            print("Erreur pour Mots-clés:" )

    infos['Mots-clés'] = mots_cles

    # Nombre de citations
    try:
        # Attendre et récupérer le nombre de citations
        infoElem = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'citation-count'))).text.split('\n')
        if len(infoElem) > 1 and infoElem[1] == 'Cited References':
            infos['Nombre de citations'] = 0  # Pas de citations
        else:
            infos['Nombre de citations'] = infoElem[0]  # Nombre de citations
    except Exception as e:
        print(f"Erreur lors de la récupération de 'citation-count': {e}")
        infos['Nombre de citations'] = None  # Valeur par défaut en cas d'erreur

    # DOI
    try:
        infos['DOI'] = wait.until(
            EC.presence_of_element_located((By.ID, 'FullRTa-DOI'))
        ).text
    except Exception as e:
        infos['DOI'] = None
        print("Erreur pour DOI:" )

    # Résumé
    try:
        infos['Résumé'] = wait.until(
            EC.presence_of_element_located((By.ID, 'FullRTa-abstract-basic'))
        ).text
    except Exception as e:
        infos['Résumé'] = None
        print("Erreur pour Résumé:" )

    # Type de document
    try:
        infos['Type de document'] = wait.until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="FullRTa-doctype-0"]'))
        ).text
    except Exception as e:
        infos['Type de document'] = None
        print("Erreur pour Type de document:" )

    try:
        # Récupérer l'ISSN de l'article
        infos['issn'] = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'value.section-label-data.text-color'))).text
    except (TimeoutException, NoSuchElementException):
        print("L'élément ISSN n'a pas été trouvé.")

    return infos


# Fonction récupère les liens des articles en parcourant les pages d'une liste d'articles.
def get_article_titles():
    titles = []
    while True:
        articles = driver.find_elements(By.CLASS_NAME, 'title')
        for article in articles:
            titles.append(article.get_attribute('href'))
        try:
            next_button = driver.find_element(By.XPATH , '//button[@data-ta="next-page-button"]')
            if 'mat-button-disabled' in next_button.get_attribute('class'):
                break
            else:
                next_button.click()
                scroll_slowly(driver, scroll_pause_time=0.2, scroll_increment=100)
        except NoSuchElementException:
            break
    return titles

# Fonction qui fait défiler lentement la page dans le navigateur.
def scroll_slowly(driver, scroll_pause_time=0.1, scroll_increment=100):

    # Récupère la hauteur totale de la page.
    last_height = driver.execute_script("return document.body.scrollHeight")
    current_scroll_position = 0  # Commence en haut de la page

    while current_scroll_position < last_height:
        # Fait défiler la page par petits incréments (scroll_increment)
        current_scroll_position += scroll_increment
        driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")

        # Attend un court instant entre les défilements
        time.sleep(scroll_pause_time)

        # Met à jour la hauteur totale au cas où la page chargerait dynamiquement plus de contenu
        last_height = driver.execute_script("return document.body.scrollHeight")


In [ ]:
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys

# Search function for journal information based on ISSN
def Search_journal_info(driver, issn):
    driver.get("https://www.scimagojr.com/")
    try:
        search_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "searchinput"))
        )
        search_input.clear()
        search_input.send_keys(issn, Keys.RETURN)

        # Wait and click on the search result
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "search_results"))
        ).click()
    except Exception as e:
        print(f"An error occurred: {e}")

# Extraction uniquement du nom du journal
# Extract publisher and journal name from the page source
def extract_publisher_and_journal_name(soup):
    publisher = None
    # Find the publisher by searching for a specific tag structure
    publisher_element = soup.find('a', href=lambda href: href and "journalsearch.php?q=" in href)
    if publisher_element:
        publisher = publisher_element.text.strip()

    return {"publisher": publisher}
# Continue with the existing functions for extracting metrics
def extract_quartile(soup):
    quartile_data = {}
    dash = soup.findAll('div', class_='dashboard')
    if dash:
        quart_dash = dash[0].findAll('div', class_="cellslide")
        if quart_dash:
            last_quartile_row = quart_dash[1].find('tbody').find_all('tr')[-1].find_all('td')
            if len(last_quartile_row) == 3:
                quartile_data = {
                    "year": last_quartile_row[1].text,
                    "quartile_value": last_quartile_row[2].text
                }
    return quartile_data

def extract_sjr(soup):
    sjr_data = {}
    dash = soup.findAll('div', class_='dashboard')
    if len(dash) > 1:
        sjr_dash = dash[1].findAll('div', class_="cellslide")[1]
        sjr_row = sjr_dash.find('tbody').find_all('tr')[-1].find_all('td')
        if len(sjr_row) == 2:
            sjr_data = {
                "year": sjr_row[0].text,
                "sjr_value": sjr_row[1].text
            }
    return sjr_data

def extract_impact_factor(soup):
    impact_data = {}
    dash = soup.findAll('div', class_='dashboard')
    if len(dash) > 1:
        impact_dash = dash[1].findAll('div', class_="cellslide")[5]
        impact_row = impact_dash.find('tbody').find_all('tr')[-1].find_all('td')
        if len(impact_row) == 3:
            impact_data = {
                "year": impact_row[1].text,
                "impact_factor_value": impact_row[2].text
            }
    return impact_data

# Main function to gather journal information
def extract_journal_metrics(driver, issn_list):
    journals_info = []

    for issn in issn_list:
        try:
            Search_journal_info(driver, issn)

            # Extract page content with BeautifulSoup
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

            # Extract journal metrics
            journal_data = {
                "issn": issn,
                "h_index": WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "hindexnumber"))
                ).text if WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "hindexnumber"))) else "N/A",
                "scope": WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "fullwidth"))
                ).text.split("\n")[1] if WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "fullwidth"))) else "N/A",
                "quartile": extract_quartile(soup),
                "sjr": extract_sjr(soup),
                "impact_factor": extract_impact_factor(soup)
            }

            # Add publisher and journal name
            journal_info = extract_publisher_and_journal_name(soup)
            journal_data.update(journal_info)

            journals_info.append(journal_data)
            print("Retrieved info for ISSN:", issn)

        except Exception as e:
            print(f"An error occurred while processing ISSN {issn}: {e}")
            journals_info.append({
                "issn": issn,
                "h_index": "N/A",
                "scope": "N/A",
                "quartile": "N/A",
                "sjr": {"sjr_value": "N/A", "year": "N/A"},
                "impact_factor": {"impact_factor_value": "N/A", "year": "N/A"},
                "publisher": "N/A"
            })

    return journals_info

In [ ]:
all_data = []  # Initialiser une liste pour stocker chaque auteur et co-auteur sans clé ID
processed_authors = set()  # Utiliser un set pour éviter les doublons d'auteurs

def fetch_author_data_with_articles(author_id):
    """Récupère les informations de l'auteur et ses articles."""
    # Récupérer les informations de l'auteur principal et les IDs de co-auteurs
    author_data, co_authors_ids = get_author_information(author_id)
    author_data['Articles'] = fetch_articles_with_journal_data(author_id)

    # Ajouter l'auteur principal à all_data si non déjà traité
    if author_id not in processed_authors:
        all_data.append(author_data)
        processed_authors.add(author_id)

    # Traiter les co-auteurs
    for co_author_id in co_authors_ids:
        if co_author_id not in processed_authors:
            co_author_data, _ = get_author_information(co_author_id)
            co_author_data['Articles'] = fetch_articles_with_journal_data(co_author_id)
            all_data.append(co_author_data)  # Ajouter le co-auteur
            processed_authors.add(co_author_id)

def fetch_articles_with_journal_data(author_id=None):
    """Récupère les articles et leurs données de journal pour un auteur donné."""
    all_articles_data = []

    # Obtenir les titres des articles
    while True:
        articles = get_article_titles()
        if articles:
            break
        driver.refresh()
        time.sleep(1)
        scroll_slowly(driver)

    if articles:
        issns = []
        for article_link in articles:
            data_article = fetch_article_data(article_link)
            all_articles_data.append(data_article)
            issns.append(data_article.get('issn'))

        # Extraire les données des journaux associés en utilisant les ISSNs des articles
        journals_data = extract_journal_metrics(driver, issns)
        for article, journal in zip(all_articles_data, journals_data):
            if article.get('issn') == journal.get('issn'):
                article['journal_data'] = journal
            article.pop('issn', None)  # Supprimer l'ISSN après association

    return all_articles_data

def fetch_article_data(article_link):
    """Extrait les détails de l'article, avec gestion des cas d'absence d'ISSN."""
    flag = 0
    while True:
        data_article = extract_article_details(driver, article_link)
        if 'issn' in data_article:
            return data_article
        else:
            flag += 1
            scroll_slowly(driver)
            if flag >= 1:
                data_article['issn'] = None
                return data_article

def main():
    for author_id in list_authors:
        if author_id not in processed_authors:
            fetch_author_data_with_articles(author_id)  # Ajouter auteur et co-auteurs

    # Enregistrer dans un fichier JSON
    with open("testo.json", "w", encoding="utf-8") as json_file:
        json.dump(all_data, json_file, indent=4, ensure_ascii=False)

    print("Données enregistrées dans le fichier 'testo.json'.")
    return all_data

# Exécution principale
try:
    all_data_result = main()
except NoSuchWindowException:
    print("La fenêtre du navigateur a été fermée. Redémarrage de la session.")

In [ ]:
# print(all_data.pop('63939275'))
print(len(all_data))
# (id_co_auteurs)

In [ ]:
import json

# Supposons que 'all_data_result' soit le résultat final de la fonction 'main()' qui contient toutes les données.
all_data_result = main()

# Enregistrer 'all_data_result' dans un fichier JSON
with open("Scrap3.json", "w", encoding="utf-8") as json_file:
    json.dump(all_data_result, json_file, ensure_ascii=False, indent=4)